# **Chapter 39: App Configuration & Development**

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Configure Flutter applications for multiple environments (development, staging, production)
- Implement Android product flavors and iOS schemes with proper build configurations
- Manage environment-specific variables and secrets securely
- Automate app icon and splash screen generation for different flavors
- Implement semantic versioning and automated build number management
- Configure CI/CD pipelines for multi-flavor builds
- Handle platform-specific configuration files (AndroidManifest.xml, Info.plist)

---

## **Prerequisites**

- Completed Chapter 38: Plugin Development (understanding of platform-specific code)
- Proficiency in Gradle configuration for Android
- Basic understanding of Xcode project configuration and schemes
- Familiarity with environment variables and build scripts
- Understanding of YAML configuration and command-line tools

---

## **39.1 Environment Configuration Strategy**

Production applications require different configurations for development, staging, and production environments. This includes API endpoints, feature flags, analytics keys, and logging levels.

### **Configuration Architecture**

```dart
// lib/config/environment.dart

import 'package:flutter/foundation.dart';

/// Enum representing application environments
enum Environment {
  /// Development environment for local testing
  /// API: Local server or mock data
  /// Logging: Verbose
  dev,
  
  /// Staging environment for QA and testing
  /// API: Staging server (production-like)
  /// Logging: Debug level
  staging,
  
  /// Production environment for end users
  /// API: Production servers
  /// Logging: Errors only
  prod,
}

/// Configuration class holding environment-specific values
/// 
/// This class uses the singleton pattern to ensure consistent
/// configuration throughout the app lifecycle
class AppConfig {
  final Environment environment;
  final String apiBaseUrl;
  final String apiKey;
  final bool enableLogging;
  final bool enableAnalytics;
  final bool enableCrashReporting;
  final Duration connectTimeout;
  final Duration receiveTimeout;
  
  // Private constructor
  const AppConfig._({
    required this.environment,
    required this.apiBaseUrl,
    required this.apiKey,
    required this.enableLogging,
    required this.enableAnalytics,
    required this.enableCrashReporting,
    required this.connectTimeout,
    required this.receiveTimeout,
  });
  
  // Singleton instance
  static late AppConfig _instance;
  
  /// Initialize configuration for the specified environment
  /// 
  /// Must be called before accessing [instance]
  static void initialize(Environment env) {
    _instance = AppConfig._(
      environment: env,
      apiBaseUrl: _getApiUrl(env),
      apiKey: _getApiKey(env),
      enableLogging: env != Environment.prod,
      enableAnalytics: true,
      enableCrashReporting: env == Environment.prod,
      connectTimeout: const Duration(seconds: 30),
      receiveTimeout: const Duration(seconds: 30),
    );
  }
  
  /// Access the current configuration
  /// 
  /// Throws [StateError] if [initialize] hasn't been called
  static AppConfig get instance {
    if (!_instanceInitialized) {
      throw StateError(
        'AppConfig not initialized. Call AppConfig.initialize() before accessing instance.'
      );
    }
    return _instance;
  }
  
  static bool get _instanceInitialized {
    try {
      _instance;
      return true;
    } catch (_) {
      return false;
    }
  }
  
  // Environment-specific configuration getters
  static String _getApiUrl(Environment env) {
    switch (env) {
      case Environment.dev:
        return 'https://api-dev.example.com/v1';
      case Environment.staging:
        return 'https://api-staging.example.com/v1';
      case Environment.prod:
        return 'https://api.example.com/v1';
    }
  }
  
  static String _getApiKey(Environment env) {
    // In production, use environment variables or secure storage
    // Never hardcode production keys in source control
    switch (env) {
      case Environment.dev:
        return 'dev-api-key-12345';
      case Environment.staging:
        return String.fromEnvironment('STAGING_API_KEY', defaultValue: 'staging-key');
      case Environment.prod:
        // Production key should come from environment variables only
        const key = String.fromEnvironment('PROD_API_KEY');
        if (key.isEmpty) {
          throw StateError('PROD_API_KEY environment variable not set');
        }
        return key;
    }
  }
  
  /// Helper to check if running in debug mode
  bool get isDebug => environment == Environment.dev;
  
  /// Helper to check if running in production
  bool get isProduction => environment == Environment.prod;
}
```

**Explanation:**

- **Singleton pattern**: Ensures only one configuration exists app-wide. Late initialization allows setting environment at app startup.
- **Environment variables**: Use `String.fromEnvironment()` to read compile-time environment variables (set during build).
- **Security**: Never commit production API keys to source control. Use CI/CD environment variables.
- **Validation**: Throw descriptive errors if configuration is accessed before initialization or if required environment variables are missing.
- **Helper getters**: `isDebug` and `isProduction` provide convenient checks for conditional logic.

### **Entry Point Configuration**

```dart
// lib/main.dart
import 'package:flutter/material.dart';
import 'config/environment.dart';

void main() {
  // Determine environment from compile-time environment variable
  // flutter run --dart-define=ENV=dev
  // flutter build apk --dart-define=ENV=prod
  const envString = String.fromEnvironment('ENV', defaultValue: 'dev');
  
  final environment = Environment.values.firstWhere(
    (e) => e.name == envString,
    orElse: () => Environment.dev,
  );
  
  // Initialize configuration before running app
  AppConfig.initialize(environment);
  
  runApp(const MyApp());
}

class MyApp extends StatelessWidget {
  const MyApp({super.key});

  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      title: 'Battery App (${AppConfig.instance.environment.name})',
      home: const BatteryScreen(),
    );
  }
}
```

**Explanation:**

- **Compile-time variables**: `--dart-define` passes values at build time, compiled into the binary. These cannot be changed at runtime.
- **Environment detection**: Parse the `ENV` variable to determine which configuration to load.
- **Early initialization**: Initialize `AppConfig` before `runApp` to ensure all widgets have access to configuration.
- **Environment indicator**: Display current environment in app title during development to avoid confusion.

---

## **39.2 Android Product Flavors**

Android product flavors allow you to build different app variants (dev, staging, prod) with different configurations, icons, and resources.

### **Gradle Configuration**

```gradle
// android/app/build.gradle

plugins {
    id "com.android.application"
    id "kotlin-android"
    id "dev.flutter.flutter-gradle-plugin"
}

android {
    namespace "com.example.battery_app"
    compileSdkVersion flutter.compileSdkVersion
    
    // Compile options for modern Java features
    compileOptions {
        sourceCompatibility JavaVersion.VERSION_1_8
        targetCompatibility JavaVersion.VERSION_1_8
    }

    kotlinOptions {
        jvmTarget = '1.8'
    }

    defaultConfig {
        applicationId "com.example.battery_app"
        minSdkVersion flutter.minSdkVersion
        targetSdkVersion flutter.targetSdkVersion
        versionCode flutterVersionCode.toInteger()
        versionName flutterVersionName
        
        // MultiDex support for large apps
        multiDexEnabled true
    }

    // SIGNING CONFIGURATIONS
    // Different keystores for different environments
    signingConfigs {
        debug {
            storeFile file('debug.keystore')
            storePassword 'android'
            keyAlias 'androiddebugkey'
            keyPassword 'android'
        }
        
        // Release signing for production
        release {
            // Load from environment variables or properties file
            // Never commit production keystore credentials to version control
            storeFile file(System.getenv("KEYSTORE_PATH") ?: "release.keystore")
            storePassword System.getenv("KEYSTORE_PASSWORD")
            keyAlias System.getenv("KEY_ALIAS")
            keyPassword System.getenv("KEY_PASSWORD")
        }
    }

    // BUILD TYPES (Debug vs Release)
    buildTypes {
        debug {
            signingConfig signingConfigs.debug
            debuggable true
            minifyEnabled false
            shrinkResources false
            // Application suffix to allow parallel installation
            applicationIdSuffix ".debug"
            versionNameSuffix "-debug"
        }
        
        release {
            signingConfig signingConfigs.release
            debuggable false
            minifyEnabled true
            shrinkResources true
            proguardFiles getDefaultProguardFile('proguard-android.txt'), 'proguard-rules.pro'
        }
        
        // Profile build for performance testing
        profile {
            initWith debug
            signingConfig signingConfigs.debug
            debuggable false
            applicationIdSuffix ".profile"
        }
    }

    // PRODUCT FLAVORS (Environment-specific configurations)
    flavorDimensions "environment"
    
    productFlavors {
        dev {
            dimension "environment"
            applicationIdSuffix ".dev"
            versionNameSuffix "-dev"
            resValue "string", "app_name", "Battery App Dev"
            
            // Build configuration fields for compile-time constants
            buildConfigField "String", "API_BASE_URL", '"https://api-dev.example.com"'
            buildConfigField "String", "ANALYTICS_KEY", '"dev-analytics-key"'
            buildConfigField "boolean", "ENABLE_LOGGING", "true"
            buildConfigField "boolean", "ENABLE_CRASH_REPORTING", "false"
        }
        
        staging {
            dimension "environment"
            applicationIdSuffix ".staging"
            versionNameSuffix "-staging"
            resValue "string", "app_name", "Battery App Staging"
            
            buildConfigField "String", "API_BASE_URL", '"https://api-staging.example.com"'
            buildConfigField "String", "ANALYTICS_KEY", '"staging-analytics-key"'
            buildConfigField "boolean", "ENABLE_LOGGING", "true"
            buildConfigField "boolean", "ENABLE_CRASH_REPORTING", "true"
        }
        
        prod {
            dimension "environment"
            // No suffix for production - this is the main app
            resValue "string", "app_name", "Battery App"
            
            buildConfigField "String", "API_BASE_URL", '"https://api.example.com"'
            buildConfigField "String", "ANALYTICS_KEY", '""' // Loaded from environment
            buildConfigField "boolean", "ENABLE_LOGGING", "false"
            buildConfigField "boolean", "ENABLE_CRASH_REPORTING", "true"
        }
    }
}

flutter {
    source '../..'
}
```

**Explanation:**

- **flavorDimensions**: Defines the dimension for flavors. You can have multiple dimensions (e.g., "environment" and "store" for Play Store vs Amazon).
- **applicationIdSuffix**: Allows installing different flavors side-by-side (e.g., `com.example.app.dev` vs `com.example.app`).
- **resValue**: Generates resources like `R.string.app_name` that can be accessed in native code or manifest.
- **buildConfigField**: Generates `BuildConfig` constants accessible in Java/Kotlin code. These are compile-time constants, not runtime variables.
- **Build types vs Flavors**: Build types (debug/release) control optimization and signing; flavors (dev/staging/prod) control features and endpoints.

### **Accessing Native Configuration from Dart**

```dart
// lib/config/native_config.dart
import 'package:flutter/services.dart';

/// Service to read native build configuration
class NativeConfig {
  static const MethodChannel _channel = 
      MethodChannel('com.example.battery_plugin/config');
  
  /// Get build configuration from native side
  static Future<Map<String, dynamic>> getBuildConfig() async {
    try {
      final Map<dynamic, dynamic> config = 
          await _channel.invokeMethod('getBuildConfig');
      return Map<String, dynamic>.from(config);
    } catch (e) {
      // Return default config if native side doesn't implement
      return {
        'environment': 'dev',
        'apiUrl': 'https://api-dev.example.com',
      };
    }
  }
  
  /// Read Android BuildConfig or iOS plist values
  static Future<String?> getNativeValue(String key) async {
    return await _channel.invokeMethod<String>('getNativeValue', {'key': key});
  }
}
```

**Explanation:**

- **Configuration bridge**: Sometimes you need to read native build configuration (like `BuildConfig.API_BASE_URL`) from Dart.
- **Graceful fallback**: If the native method isn't implemented, return sensible defaults rather than crashing.
- **Type casting**: Platform channels return `dynamic`, so cast to specific types carefully.

---

## **39.3 iOS Schemes and Configurations**

iOS uses Xcode schemes and build configurations to manage different environments, analogous to Android flavors.

### **Xcode Configuration**

In `ios/Runner.xcworkspace`, configure build settings:

```ruby
# ios/Runner/Info.plist additions for environment identification
# Add to Info.plist:
# <key>Environment</key>
# <string>$(ENVIRONMENT)</string>
# <key>ApiBaseUrl</key>
# <string>$(API_BASE_URL)</string>
```

**Explanation:**

- **User-Defined Settings**: In Xcode, you can define custom build settings like `ENVIRONMENT` and `API_BASE_URL` that get substituted into Info.plist at build time.
- **Scheme-based configuration**: Each Xcode scheme (Debug, Release, Profile) can have different build configurations, but for flavors, you typically create additional configurations like "Debug-dev", "Release-dev", etc.

### **Reading iOS Configuration in Swift**

```swift
// ios/Runner/AppDelegate.swift additions
import Flutter

@UIApplicationMain
@objc class AppDelegate: FlutterAppDelegate {
  override func application(
    _ application: UIApplication,
    didFinishLaunchingWithOptions launchOptions: [UIApplication.LaunchOptionsKey: Any]?
  ) -> Bool {
    
    let controller = window?.rootViewController as! FlutterViewController
    
    // Setup method channel for configuration
    let configChannel = FlutterMethodChannel(
      name: "com.example.app/config",
      binaryMessenger: controller.binaryMessenger
    )
    
    configChannel.setMethodCallHandler { (call, result) in
      if call.method == "getBuildConfig" {
        // Read from Info.plist
        let bundle = Bundle.main
        let environment = bundle.object(forInfoDictionaryKey: "Environment") as? String ?? "unknown"
        let apiUrl = bundle.object(forInfoDictionaryKey: "ApiBaseUrl") as? String ?? ""
        
        let config: [String: Any] = [
          "environment": environment,
          "apiUrl": apiUrl,
          "platform": "ios"
        ]
        result(config)
      } else {
        result(FlutterMethodNotImplemented)
      }
    }
    
    GeneratedPluginRegistrant.register(with: self)
    return super.application(application, didFinishLaunchingWithOptions: launchOptions)
  }
}
```

**Explanation:**

- **Info.plist access**: Use `Bundle.main.object(forInfoDictionaryKey:)` to read values from Info.plist that were set via build settings.
- **Method channel setup**: Configure the channel in `AppDelegate` before `GeneratedPluginRegistrant.register` to ensure it's available when Flutter starts.

---

## **39.4 Automated Asset Generation**

Different flavors require different app icons, splash screens, and app names. Automating this prevents manual errors.

### **Flutter Launcher Icons Configuration**

```yaml
# pubspec.yaml
dev_dependencies:
  flutter_launcher_icons: ^0.13.1

flutter_launcher_icons:
  # Default configuration (fallback)
  image_path: "assets/icon/icon.png"
  
  # Android specific
  android: true
  adaptive_icon_background: "assets/icon/background.png"
  adaptive_icon_foreground: "assets/icon/foreground.png"
  
  # iOS specific
  ios: true
  remove_alpha_ios: true
  
  # Flavor-specific configurations
  flavors:
    dev:
      image_path: "assets/icon/dev_icon.png"
      android:
        adaptive_icon_background: "assets/icon/dev_background.png"
      ios:
        image_path: "assets/icon/dev_icon.png"
    
    staging:
      image_path: "assets/icon/staging_icon.png"
    
    prod:
      image_path: "assets/icon/prod_icon.png"
```

**Explanation:**

- **flutter_launcher_icons**: Dev dependency that generates all required icon sizes for Android and iOS from source images.
- **Adaptive icons**: Android 8.0+ uses adaptive icons with separate background and foreground layers.
- **Flavor-specific assets**: The `flavors` section allows different icons for each flavor, making it visually obvious which version is installed.
- **Generation command**: `flutter pub run flutter_launcher_icons:main -f dev` generates icons for the dev flavor.

### **Native Splash Screen Configuration**

```yaml
# pubspec.yaml
dev_dependencies:
  flutter_native_splash: ^2.3.0

flutter_native_splash:
  # Default configuration
  color: "#42a5f5"
  image: assets/splash/splash.png
  branding: assets/splash/branding.png
  branding_mode: bottom
  
  # Android 12+ specific
  android_12:
    image: assets/splash/android12_splash.png
    icon_background_color: "#42a5f5"
    
  # Flavor-specific configurations
  flavors:
    dev:
      color: "#ff5722"  # Orange for dev
      image: assets/splash/dev_splash.png
      
    staging:
      color: "#ffc107"  # Yellow for staging
      
    prod:
      color: "#4caf50"  # Green for production
```

**Explanation:**

- **flutter_native_splash**: Generates native splash screens that display immediately when the app launches, before Flutter renders its first frame.
- **Android 12+**: Android 12 introduced new splash screen APIs with different requirements (must use windowSplashScreenAnimatedIcon).
- **Branding**: Optional bottom branding image for additional logos or text.
- **Flavor colors**: Different background colors make it immediately obvious which environment you're launching.

---

## **39.5 Build Scripts and Automation**

Automating flavor builds ensures consistency and prevents manual configuration errors.

### **Build Runner Script**

```dart
// tools/build_runner.dart
import 'dart:io';
import 'package:args/args.dart';

/// Build runner script for automating flavor builds
/// 
/// Usage: dart tools/build_runner.dart --flavor prod --platform android
void main(List<String> arguments) {
  final parser = ArgParser()
    ..addOption('flavor', 
        abbr: 'f', 
        allowed: ['dev', 'staging', 'prod'],
        defaultsTo: 'dev',
        help: 'Build flavor')
    ..addOption('platform',
        abbr: 'p',
        allowed: ['android', 'ios', 'all'],
        defaultsTo: 'all',
        help: 'Target platform')
    ..addFlag('release',
        abbr: 'r',
        defaultsTo: false,
        help: 'Build release version')
    ..addFlag('install',
        abbr: 'i',
        defaultsTo: false,
        help: 'Install after building');

  final results = parser.parse(arguments);
  final flavor = results['flavor'] as String;
  final platform = results['platform'] as String;
  final isRelease = results['release'] as bool;
  final shouldInstall = results['install'] as bool;

  print('🔨 Building flavor: $flavor');
  print('📱 Platform: $platform');
  print('🚀 Release mode: $isRelease');

  // Pre-build steps
  _generateAssets(flavor);
  _validateConfig(flavor);

  // Build commands
  if (platform == 'android' || platform == 'all') {
    _buildAndroid(flavor, isRelease, shouldInstall);
  }
  
  if (platform == 'ios' || platform == 'all') {
    _buildiOS(flavor, isRelease, shouldInstall);
  }

  print('✅ Build completed successfully!');
}

void _generateAssets(String flavor) {
  print('🎨 Generating assets for $flavor...');
  
  // Generate launcher icons
  _runCommand('flutter', [
    'pub', 'run', 'flutter_launcher_icons:main',
    '-f', flavor
  ]);
  
  // Generate splash screen
  _runCommand('flutter', [
    'pub', 'run', 'flutter_native_splash:create',
    '--flavor', flavor
  ]);
}

void _validateConfig(String flavor) {
  print('🔍 Validating configuration...');
  
  // Check if required environment variables are set for production
  if (flavor == 'prod') {
    const apiKey = String.fromEnvironment('PROD_API_KEY');
    if (apiKey.isEmpty) {
      throw Exception('PROD_API_KEY environment variable must be set for production builds');
    }
  }
}

void _buildAndroid(String flavor, bool isRelease, bool install) {
  print('🤖 Building Android ${isRelease ? "APK/AAB" : "debug"}...');
  
  final args = [
    'build',
    isRelease ? 'appbundle' : 'apk',
    '--flavor', flavor,
    if (!isRelease) '--debug',
  ];
  
  _runCommand('flutter', args);
  
  if (install && !isRelease) {
    _runCommand('flutter', ['install', '--flavor', flavor]);
  }
}

void _buildiOS(String flavor, bool isRelease, bool install) {
  print('🍎 Building iOS...');
  
  // iOS uses schemes instead of flavors
  final scheme = flavor == 'prod' ? 'Runner' : 'Runner $flavor';
  
  _runCommand('flutter', [
    'build',
    'ios',
    '--flavor', flavor,
    if (isRelease) '--release' else '--debug',
  ]);
}

void _runCommand(String executable, List<String> arguments) {
  print('Running: $executable ${arguments.join(' ')}');
  final result = Process.runSync(executable, arguments);
  
  if (result.exitCode != 0) {
    print('Error: ${result.stderr}');
    throw Exception('Command failed: $executable ${arguments.join(' ')}');
  }
  
  print(result.stdout);
}
```

**Explanation:**

- **Args package**: Use `package:args` for robust command-line argument parsing with help text and validation.
- **Asset generation**: Automate icon and splash generation for the specific flavor before building.
- **Environment validation**: Check that required environment variables exist before attempting production builds.
- **Flavor mapping**: Android uses `--flavor`, iOS uses schemes. The script abstracts these differences.
- **Process runner**: Helper to run shell commands and capture output, failing fast on errors.

---

## **39.6 iOS Schemes and Configurations**

iOS uses Xcode schemes and build configurations to manage different environments, analogous to Android flavors but requiring different setup.

### **Xcode Configuration Setup**

```ruby
# ios/Runner.xcodeproj/project.pbxproj configuration (managed via Xcode IDE or CocoaPods)
# The following represents the conceptual setup:

# Build Configurations needed:
# - Debug-dev, Debug-staging, Debug-prod
# - Release-dev, Release-staging, Release-prod
# - Profile-dev, Profile-staging, Profile-prod

# Schemes needed:
# - Runner (prod)
# - Runner-dev
# - Runner-staging
```

**Explanation:**

- **Build Configurations**: iOS uses configurations (Debug/Release/Profile) combined with user-defined settings to create flavor-like behavior.
- **Schemes**: Xcode schemes combine a build configuration with a target. Create separate schemes for each flavor to easily switch in Xcode.
- **Configuration files**: Use `.xcconfig` files to manage settings per environment.

### **XCConfig Files**

Create configuration files for each environment:

```ini
# ios/Flutter/dev.xcconfig
#include "Generated.xcconfig"
#include "Pods/Target Support Files/Pods-Runner/Pods-Runner.debug-dev.xcconfig"

FLUTTER_FLAVOR=dev
APP_DISPLAY_NAME=Battery App Dev
API_BASE_URL=https://api-dev.example.com
BUNDLE_ID_SUFFIX=.dev
```

```ini
# ios/Flutter/prod.xcconfig
#include "Generated.xcconfig"
#include "Pods/Target Support Files/Pods-Runner/Pods-Runner.release-prod.xcconfig"

FLUTTER_FLAVOR=prod
APP_DISPLAY_NAME=Battery App
API_BASE_URL=https://api.example.com
BUNDLE_ID_SUFFIX=
```

**Explanation:**

- **#include**: XCConfig files can include other configs, allowing you to layer settings.
- **User-defined settings**: `FLUTTER_FLAVOR` and custom keys can be read in Info.plist using the `${VAR}` syntax.
- **Bundle ID suffix**: Append `.dev` or `.staging` to bundle identifiers to allow parallel installation.

### **Info.plist Configuration**

```xml
<!-- ios/Runner/Info.plist -->
<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE plist PUBLIC "-//Apple//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<plist version="1.0">
<dict>
    <!-- Standard Flutter entries -->
    <key>CFBundleDevelopmentRegion</key>
    <string>$(DEVELOPMENT_LANGUAGE)</string>
    <key>CFBundleDisplayName</key>
    <string>$(APP_DISPLAY_NAME)</string>
    <key>CFBundleIdentifier</key>
    <string>$(PRODUCT_BUNDLE_IDENTIFIER)$(BUNDLE_ID_SUFFIX)</string>
    
    <!-- Custom environment variables -->
    <key>Environment</key>
    <string>$(FLUTTER_FLAVOR)</string>
    <key>ApiBaseUrl</key>
    <string>$(API_BASE_URL)</string>
    
    <!-- Privacy descriptions required for App Store -->
    <key>NSCameraUsageDescription</key>
    <string>This app needs camera access to scan QR codes</string>
</dict>
</plist>
```

**Explanation:**

- **Variable substitution**: Xcode replaces `$(VAR)` syntax with values from the active XCConfig file at build time.
- **Bundle identifier**: Append suffixes to allow parallel installation (e.g., `com.example.app` and `com.example.app.dev`).
- **Display name**: Change the app name shown on the home screen for each flavor.
- **Privacy keys**: iOS 14+ requires descriptions for sensitive permissions in Info.plist.

### **Reading iOS Configuration in Flutter**

```dart
// lib/config/ios_config.dart
import 'dart:io';
import 'package:flutter/services.dart';

/// Service to read iOS-specific configuration from Info.plist
class IosConfigReader {
  static const MethodChannel _channel = 
      MethodChannel('com.example.app/config');
  
  /// Read a value from Info.plist
  static Future<String?> readPlistValue(String key) async {
    if (!Platform.isIOS) return null;
    
    try {
      return await _channel.invokeMethod<String>('readPlistValue', {
        'key': key,
      });
    } catch (e) {
      print('Error reading plist value: $e');
      return null;
    }
  }
  
  /// Get current environment from Info.plist
  static Future<String> getEnvironment() async {
    return await readPlistValue('Environment') ?? 'unknown';
  }
  
  /// Get API base URL from native config
  static Future<String> getApiBaseUrl() async {
    return await readPlistValue('ApiBaseUrl') ?? 'https://api-dev.example.com';
  }
}
```

**Explanation:**

- **Platform checks**: Only attempt to read iOS configuration when running on iOS.
- **Method channel**: Communicate with native iOS code to read Info.plist values at runtime.
- **Fallback values**: Provide sensible defaults if native configuration is unavailable.

---

## **39.4 Versioning and Build Automation**

Consistent versioning across platforms is crucial for release management and debugging.

### **Semantic Versioning in Flutter**

```yaml
# pubspec.yaml
name: battery_app
description: A battery monitoring application
publish_to: 'none'
version: 1.2.3+15  # versionName+versionCode
```

**Explanation:**

- **versionName (1.2.3)**: Semantic version shown to users. Format: `MAJOR.MINOR.PATCH`
  - MAJOR: Breaking changes
  - MINOR: New features, backward compatible
  - PATCH: Bug fixes
- **versionCode (+15)**: Integer build number used by app stores to determine if one version is newer than another. Must increment for each release.

### **Automated Version Management**

```yaml
# pubspec.yaml
dev_dependencies:
  cider: ^0.2.0  # Version management tool
  change_app_package_name: ^1.1.0
```

```bash
# Version bumping commands
cider bump major  # 1.2.3 -> 2.0.0
cider bump minor  # 1.2.3 -> 1.3.0
cider bump patch  # 1.2.3 -> 1.2.4
cider bump build  # +15 -> +16
cider version      # Display current version
```

**Explanation:**

- **Cider**: Dart package for managing semantic versioning in pubspec.yaml automatically.
- **Build number**: Use `cider bump build` to increment the versionCode before each release build.
- **Consistency**: Ensures version numbers follow semver strictly without manual editing errors.

### **Synchronizing Version Across Platforms**

```dart
// tools/version_sync.dart
import 'dart:io';
import 'package:yaml/yaml.dart';

/// Synchronizes version numbers across Android and iOS native projects
/// based on the version defined in pubspec.yaml
void main() {
  final pubspecFile = File('pubspec.yaml');
  final pubspecContent = pubspecFile.readAsStringSync();
  final pubspec = loadYaml(pubspecContent);
  
  final version = pubspec['version'] as String;
  final parts = version.split('+');
  final versionName = parts[0];  // e.g., "1.2.3"
  final versionCode = parts[1]; // e.g., "15"
  
  print('Syncing version $versionName ($versionCode) to native projects...');
  
  _updateAndroidBuildGradle(versionName, versionCode);
  _updateIosProject(versionName, versionCode);
  
  print('Version sync complete!');
}

void _updateAndroidBuildGradle(String versionName, String versionCode) {
  final file = File('android/app/build.gradle');
  if (!file.existsSync()) {
    print('Warning: android/app/build.gradle not found');
    return;
  }
  
  var content = file.readAsStringSync();
  
  // Replace versionCode
  content = content.replaceAllMapped(
    RegExp(r'versionCode\s+(flutterVersionCode\.toInteger\(\)|\d+)'),
    (match) => 'versionCode $versionCode',
  );
  
  // Replace versionName
  content = content.replaceAllMapped(
    RegExp(r'versionName\s+(flutterVersionName|"[0-9.]+")'),
    (match) => 'versionName "$versionName"',
  );
  
  file.writeAsStringSync(content);
  print('Updated Android build.gradle');
}

void _updateIosProject(String versionName, String versionCode) {
  // Update Info.plist
  final infoPlist = File('ios/Runner/Info.plist');
  if (infoPlist.existsSync()) {
    var content = infoPlist.readAsStringSync();
    
    // Update CFBundleShortVersionString (version name)
    content = content.replaceAllMapped(
      RegExp(r'(<key>CFBundleShortVersionString</key>\s*<string>)[^<]+'),
      (match) => '${match.group(1)}$versionName',
    );
    
    // Update CFBundleVersion (build number)
    content = content.replaceAllMapped(
      RegExp(r'(<key>CFBundleVersion</key>\s*<string>)[^<]+'),
      (match) => '${match.group(1)}$versionCode',
    );
    
    infoPlist.writeAsStringSync(content);
    print('Updated iOS Info.plist');
  }
}
```

**Explanation:**

- **Version synchronization**: Native projects (Android/iOS) need their version numbers updated to match pubspec.yaml for app store compliance.
- **Regex replacement**: Carefully update version strings in Gradle and plist files without breaking file structure.
- **CI/CD integration**: Run this script in CI pipelines before building to ensure version consistency.

---

## **Chapter Summary**

In this chapter, we covered comprehensive app configuration and flavor management:

### **Key Takeaways:**

1. **Environment Strategy**: Use compile-time variables (`--dart-define`) for environment-specific configuration (API URLs, feature flags) and secure storage for secrets.

2. **Android Product Flavors**: Configure in `build.gradle` with `flavorDimensions` and `productFlavors`, using `applicationIdSuffix` for parallel installation and `buildConfigField` for compile-time constants.

3. **iOS Schemes**: Use Xcode build configurations and `.xcconfig` files to manage environment-specific settings, reading values from Info.plist in native code.

4. **Federated Plugin Configuration**: When building plugins, use platform interfaces to allow apps to read native configuration consistently across platforms.

5. **Asset Generation**: Automate icon and splash screen generation per flavor using `flutter_launcher_icons` and `flutter_native_splash` to ensure visual distinction between environments.

6. **Version Management**: Use semantic versioning (semver) with automated tools like `cider`, synchronizing version codes across Android (build.gradle) and iOS (Info.plist) before release builds.

7. **Security**: Never commit production keys to source control. Use environment variables in CI/CD and native build configurations for secrets.

### **Best Practices Checklist:**
- ✅ Use different app icons for each flavor to prevent confusion
- ✅ Implement proper error handling when reading native configuration
- ✅ Keep environment-specific API URLs in native build config, not Dart code
- ✅ Version sync script in CI/CD to ensure native version codes match pubspec
- ✅ Use `applicationIdSuffix` on Android to allow dev/staging/prod side-by-side installation
- ✅ Document required environment variables in README for team onboarding

---

## **Next Steps**

In the next chapter, **Chapter 40: Internationalization (i18n)**, we will explore how to make your Flutter applications accessible to global audiences. You'll learn how to manage translations using ARB files, implement locale resolution, handle RTL (Right-to-Left) languages, format dates/numbers according to locale conventions, and manage pluralization and gender-specific translations.

---

**End of Chapter 39**

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../10. Advanced_topics/38. plugin_development.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='40. internationalization.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
